In [18]:
import pandas as pd 
import numpy as np

In [3]:
train_d = pd.read_csv('aapl_5m_train.csv')
validation_d = pd.read_csv('aapl_5m_validation.csv')

In [4]:
#1 Select strategies
#2 Trade Signals(Buy/sell)
#3 Backtesting-> Portafolio Val
#4 Max Profit (SL 1-5%, TP 1-5%, PL1 1-50, PL2 51,200)
#5 Params 
#6 Test all possible combinations
#7 Select optimal strategy
#8 Backtest with data

In [29]:
# Strategies
n = list(range(1,2**5))
combinations = list(map(lambda x: [int(bit) for bit in f"{x:05b}"], n))

from typing import List
def minimize_port(x: np.array, byte_array: List[int]):
    sl, tp, *strat_args = x
    v = backtest(byte_array, sl, tp, strat_args)
    return -v

# Checar a partir esto
def backtest(strat,sl,tp,*strat_args):
    strategy_signals = gen_signals(strat,signal_type:str,*strat_args)
    
    for time, row in strategy_signals,iterations()
        if row.sum() = len(row)
        
def gen_signals(strat:list[int],signal_type:str,*args):
    ct = 0
    strat_args = args
    macd,sma,rsi = strat
    
    if macd:
        ct += strat_dict["macd"]["num_params"]
        ...
    if sma:
        long = strat_args[ct]
        short = strat_args[ct+1]
        ct +=
            if signal_type == "BUY":
                df["sma"]

SyntaxError: invalid syntax (Temp/ipykernel_724/4073730830.py, line 16)